# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:

import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from os.path import dirname

#### Creating list of filepaths to process original event csv data files

In [2]:

# get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data/'
print(filepath)
print(list(os.walk(filepath)))


for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))


/home/workspace/event_data/
[('/home/workspace/event_data/', [], ['2018-11-30-events.csv', '2018-11-23-events.csv', '2018-11-22-events.csv', '2018-11-29-events.csv', '2018-11-11-events.csv', '2018-11-14-events.csv', '2018-11-20-events.csv', '2018-11-15-events.csv', '2018-11-05-events.csv', '2018-11-28-events.csv', '2018-11-25-events.csv', '2018-11-16-events.csv', '2018-11-18-events.csv', '2018-11-24-events.csv', '2018-11-04-events.csv', '2018-11-19-events.csv', '2018-11-26-events.csv', '2018-11-12-events.csv', '2018-11-27-events.csv', '2018-11-06-events.csv', '2018-11-09-events.csv', '2018-11-03-events.csv', '2018-11-21-events.csv', '2018-11-07-events.csv', '2018-11-01-events.csv', '2018-11-13-events.csv', '2018-11-17-events.csv', '2018-11-08-events.csv', '2018-11-10-events.csv', '2018-11-02-events.csv'])]
30


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
    #print(f)

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


/home/workspace/event_data/2018-11-30-events.csv
/home/workspace/event_data/2018-11-23-events.csv
/home/workspace/event_data/2018-11-22-events.csv
/home/workspace/event_data/2018-11-29-events.csv
/home/workspace/event_data/2018-11-11-events.csv
/home/workspace/event_data/2018-11-14-events.csv
/home/workspace/event_data/2018-11-20-events.csv
/home/workspace/event_data/2018-11-15-events.csv
/home/workspace/event_data/2018-11-05-events.csv
/home/workspace/event_data/2018-11-28-events.csv
/home/workspace/event_data/2018-11-25-events.csv
/home/workspace/event_data/2018-11-16-events.csv
/home/workspace/event_data/2018-11-18-events.csv
/home/workspace/event_data/2018-11-24-events.csv
/home/workspace/event_data/2018-11-04-events.csv
/home/workspace/event_data/2018-11-19-events.csv
/home/workspace/event_data/2018-11-26-events.csv
/home/workspace/event_data/2018-11-12-events.csv
/home/workspace/event_data/2018-11-27-events.csv
/home/workspace/event_data/2018-11-06-events.csv
/home/workspace/even

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster()
    session = cluster.connect()
except Exception as e:
    print(e)


#### Create Keyspace

In [6]:

try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS music_db
        WITH REPLICATION =
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
        """)
except Exception as e:
    print(e)


#### Set Keyspace

In [7]:
try:
    session.set_keyspace('music_db')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




# show the dataframe

In [8]:
df = pd.read_csv('event_datafile_new.csv', encoding='utf8')
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Stephen Lynch,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",829,Jim Henson's Dead,91
1,Manowar,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Shell Shock,73
2,Morcheeba,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Women Lose Weight (Feat: Slick Rick),73
3,Maroon 5,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Won't Go Home Without You,73
4,Train,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Hey_ Soul Sister,73


# helper functions 
create_or_drop_table creates or drops tables with query as arg
select_rows select rows with query as arg

In [9]:
def create_or_drop_table(query):
    try:
        session.execute(query)
    except Exception as e:
        print(e)

In [10]:
def select_rows(query):
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)
    
    for row in rows:
        print (list(row))
    

## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during 
## sessionId = 338, and itemInSession = 4

Primary key: sessionId and itemInSession

Partition key: sessionId



In [11]:


create_or_drop_table("""
            CREATE TABLE IF NOT EXISTS music_history (
            session_id INT, 
            itemInSession INT, 
            artist TEXT, 
            song TEXT, 
            length FLOAT, 
            PRIMARY KEY(session_id, itemInSession)
            )""")

                    

In [12]:
for i, row in df.iterrows():
    query  = "INSERT INTO music_history (session_id, itemInSession, artist, song, length)"
    query += "VALUES(%s, %s, %s, %s, %s)"
    session.execute(query, (row.sessionId, row.itemInSession, row.artist, row.song, row.length))


#### SELECT to verify that the data have been inserted into each table

In [13]:
select_rows("""SELECT 
                artist, 
                song, 
                length 
            FROM music_history 
            WHERE session_id=338 AND itemInSession=4
            """)


['Faithless', 'Music Matters (Mark Knight Dub)', 495.30731201171875]


### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

Partition key: userId

Primary key: user_id, sessionId, itemInSession


In [14]:
create_or_drop_table("""CREATE TABLE IF NOT EXISTS user_history(
                            user_id INT,
                            sessionId INT,
                            itemInSession INT,  
                            artist TEXT, 
                            song TEXT, 
                            firstName TEXT, 
                            lastName TEXT,
                   PRIMARY KEY(user_id, sessionId, itemInSession))""")


for i, row in df.iterrows():
    query  = "INSERT INTO user_history (user_id, sessionId, itemInSession, artist, song, firstName, lastName)"
    query += "VALUES(%s, %s, %s, %s, %s, %s, %s)"
    session.execute(query, (row.userId, row.sessionId, row.itemInSession, row.artist, row.song, row.firstName, row.lastName))
                    

select_rows("""SELECT 
                    itemInSession, 
                    artist, 
                    song, 
                    firstName,
                    lastName
                FROM user_history 
                WHERE user_id=10 AND sessionId=182""")


[0, 'Down To The Bone', "Keep On Keepin' On", 'Sylvie', 'Cruz']
[1, 'Three Drives', 'Greece 2000', 'Sylvie', 'Cruz']
[2, 'Sebastien Tellier', 'Kilometer', 'Sylvie', 'Cruz']
[3, 'Lonnie Gordon', 'Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', 'Sylvie', 'Cruz']


# Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Partition key: song

Primary key: song, userId

In [18]:
create_or_drop_table("""CREATE TABLE IF NOT EXISTS song_history(
                    song TEXT, 
                    userId INT,
                    firstName TEXT, 
                    lastName TEXT,
           PRIMARY KEY(song, userId)
           )""")

    
for i, row in df.iterrows():
    query  = "INSERT INTO song_history (song, userId, firstName, lastName)"
    query += "VALUES(%s, %s, %s, %s)"
    session.execute(query, (row.song, row.userId, row.firstName, row.lastName))
    
select_rows("""SELECT   userId,
                        firstName,
                        lastName
                FROM song_history 
                WHERE song='All Hands Against His Own'""")

    

                    

[29, 'Jacqueline', 'Lynch']
[80, 'Tegan', 'Levine']
[95, 'Sara', 'Johnson']


### Drop the tables before closing out the sessions

In [20]:
session.execute("DROP TABLE IF EXISTS music_history")
session.execute("DROP TABLE IF EXISTS user_history")
session.execute("DROP TABLE IF EXISTS song_history")


### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()